In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re

In [2]:
df = pd.read_csv('../data/bayut/ready_flats.csv', converters={'amenities': literal_eval})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29108 entries, 0 to 29107
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   URL           29108 non-null  object 
 1   listing_id    29108 non-null  int64  
 2   type          29108 non-null  object 
 3   building      15944 non-null  object 
 4   district      29108 non-null  object 
 5   neighborhood  28750 non-null  object 
 6   price         29108 non-null  int64  
 7   beds          25683 non-null  float64
 8   baths         29108 non-null  object 
 9   surface       29108 non-null  float64
 10  prop_area     29108 non-null  float64
 11  lat           29108 non-null  float64
 12  long          29108 non-null  float64
 13  highlights    29108 non-null  object 
 14  condition     29108 non-null  object 
 15  purpose       29108 non-null  object 
 16  desc          29108 non-null  object 
 17  completion    29108 non-null  object 
 18  furnishing    17154 non-nu

In [3]:
df['baths'] = df['baths'].apply(lambda x: x.replace('[','').replace(']','').replace("'",''))
df['highlights'] = df['highlights'].apply(lambda x: x.replace('|',',').replace(' ,',','))
df[['baths', 'beds']] = df[['baths', 'beds']].apply(pd.to_numeric, errors='coerce')
df.set_index('listing_id', inplace=True)

df2 = df[(df['completion'] == 'Ready')]
df2 = df2[['URL', 'building', 'district', 'neighborhood', 'price', 'beds', 'baths', \
          'surface', 'lat', 'long', 'highlights', 'furnishing', 'amenities']]
df2 = df2[~df2.index.duplicated(keep='first')]

In [4]:
df2.head()

,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,highlights,furnishing,amenities
listing_id,,,,,,,,,,,,,
5825610,https://www.bayut.com/property/details-5825610...,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,1,815.0,25.177226,55.308851,"Vacant Now, Spacious Layout, 1 Bedroom",NaN,[Balcony or Terrace]
5933491,https://www.bayut.com/property/details-5933491...,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,2,1141.0,25.060612,55.200228,"Kitchen Appliances, Pool View, Best Investment",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi..."
5933545,https://www.bayut.com/property/details-5933545...,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,2,618.0,25.064736,55.207565,"Brand New, Good Investment, Vacant And Ready ...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi..."
5933677,https://www.bayut.com/property/details-5933677...,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,2,933.0,25.063390,55.202050,"Unfurnished 1BHK + Study Room, Private Garden,...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi..."
5929393,https://www.bayut.com/property/details-5929393...,Golden Mile 6,Palm Jumeirah,Golden Mile,2500000,2.0,2,1794.0,25.109089,55.144157,"Exclusive, Sea & Park View, Upgraded",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi..."


In [5]:
amenities = df2[(df2['amenities'] != '[]')]['amenities']

a_list = []

for amenity in amenities:
    if amenity:
        for a in amenity:
            a_list.append(a)
            
unique = list(dict.fromkeys(a_list))
print(unique)

['Balcony or Terrace', 'Parking Spaces: 1', 'Swimming Pool', 'Electricity Backup', 'Centrally Air-Conditioned', 'Storage Areas', 'Lobby in Building', 'Reception/Waiting Room', 'Gym or Health Club', 'Barbeque Area', 'Study Room', 'Service Elevators', 'Waste Disposal', 'Business Center', 'Laundry Room', 'Intercom', '24 Hours Concierge', 'Flooring', 'Central Heating', 'Kids Play Area', 'Maintenance Staff', 'Cleaning Services', 'Security Staff', 'CCTV Security', 'Parking Spaces', 'Jacuzzi', 'Sauna', 'Double Glazed Windows', 'Completion Year: 2007', 'Steam Room', 'Lawn or Garden', 'Laundry Facility', 'Broadband Internet', 'Satellite/Cable TV', 'Elevators in Building: 5', 'Prayer Room', 'First Aid Medical Center', 'Facilities for Disabled', 'Cafeteria or Canteen', 'ATM Facility', 'Elevators in Building', 'Completion Year: 2006', 'Furnished', 'Maids Room', 'View', 'Parking Spaces: 2', 'Conference Room', 'Freehold', 'Completion Year: 2008', 'Day Care Center', 'Elevators in Building: 2', 'Eleva

In [6]:
# Completion Year:

completion_years = []

def get_years():
    for lists in df2['amenities']:
        str = ','.join(lists)
        m = re.search('Completion Year: (\d{4})', str)
        if m:
            completion_years.append(m.group(1))
        else:
            completion_years.append(0)
            
get_years()

print(len(completion_years))
        
df2['completion_year'] = completion_years
df2

29065


,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,highlights,furnishing,amenities,completion_year
listing_id,,,,,,,,,,,,,,
5825610,https://www.bayut.com/property/details-5825610...,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,1,815.0,25.177226,55.308851,"Vacant Now, Spacious Layout, 1 Bedroom",NaN,[Balcony or Terrace],0
5933491,https://www.bayut.com/property/details-5933491...,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,2,1141.0,25.060612,55.200228,"Kitchen Appliances, Pool View, Best Investment",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0
5933545,https://www.bayut.com/property/details-5933545...,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,2,618.0,25.064736,55.207565,"Brand New, Good Investment, Vacant And Ready ...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0
5933677,https://www.bayut.com/property/details-5933677...,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,2,933.0,25.063390,55.202050,"Unfurnished 1BHK + Study Room, Private Garden,...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0
5929393,https://www.bayut.com/property/details-5929393...,Golden Mile 6,Palm Jumeirah,Golden Mile,2500000,2.0,2,1794.0,25.109089,55.144157,"Exclusive, Sea & Park View, Upgraded",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4659412,https://www.bayut.com/property/details-4659412...,NaN,Dubai Residence Complex,Skycourts Towers,369000,1.0,2,734.0,25.091303,55.385476,"New listi0ng, No Commission, High Floor, Vacant",Unfurnished,"[Balcony or Terrace, Parking Spaces, Swimming ...",2010
4513630,https://www.bayut.com/property/details-4513630...,Jumeirah Living,World Trade Centre,World Trade Centre Residence,1850960,2.0,2,1851.0,25.226713,55.289187,"Exclusive, Luxurious Duplex, Open Kitchen",NaN,"[Floor, Steam Room, View, Centrally Air-Condit...",0
4408791,https://www.bayut.com/property/details-4408791...,Icon Tower 1,Jumeirah Lake Towers (JLT),Icon Tower,900000,2.0,3,1098.0,25.064754,55.138866,"Covered Parking, Store & Laundry Room, Wardrobes",Unfurnished,"[Balcony or Terrace, Parking Spaces, Swimming ...",2009


In [7]:
# Floor:

floors = []

def get_floors():
    for lists in df2['amenities']:
        str = ','.join(lists)
        m = re.search('Floor: (\d+)', str)
        if m:
            floors.append(m.group(1))
        else:
            floors.append('')
            
get_floors()

print(len(floors))
        
df2['floor'] = floors
df2

29065


,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,highlights,furnishing,amenities,completion_year,floor
listing_id,,,,,,,,,,,,,,,
5825610,https://www.bayut.com/property/details-5825610...,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,1,815.0,25.177226,55.308851,"Vacant Now, Spacious Layout, 1 Bedroom",NaN,[Balcony or Terrace],0,
5933491,https://www.bayut.com/property/details-5933491...,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,2,1141.0,25.060612,55.200228,"Kitchen Appliances, Pool View, Best Investment",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,
5933545,https://www.bayut.com/property/details-5933545...,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,2,618.0,25.064736,55.207565,"Brand New, Good Investment, Vacant And Ready ...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,
5933677,https://www.bayut.com/property/details-5933677...,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,2,933.0,25.063390,55.202050,"Unfurnished 1BHK + Study Room, Private Garden,...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,
5929393,https://www.bayut.com/property/details-5929393...,Golden Mile 6,Palm Jumeirah,Golden Mile,2500000,2.0,2,1794.0,25.109089,55.144157,"Exclusive, Sea & Park View, Upgraded",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4659412,https://www.bayut.com/property/details-4659412...,NaN,Dubai Residence Complex,Skycourts Towers,369000,1.0,2,734.0,25.091303,55.385476,"New listi0ng, No Commission, High Floor, Vacant",Unfurnished,"[Balcony or Terrace, Parking Spaces, Swimming ...",2010,
4513630,https://www.bayut.com/property/details-4513630...,Jumeirah Living,World Trade Centre,World Trade Centre Residence,1850960,2.0,2,1851.0,25.226713,55.289187,"Exclusive, Luxurious Duplex, Open Kitchen",NaN,"[Floor, Steam Room, View, Centrally Air-Condit...",0,
4408791,https://www.bayut.com/property/details-4408791...,Icon Tower 1,Jumeirah Lake Towers (JLT),Icon Tower,900000,2.0,3,1098.0,25.064754,55.138866,"Covered Parking, Store & Laundry Room, Wardrobes",Unfurnished,"[Balcony or Terrace, Parking Spaces, Swimming ...",2009,


In [8]:
df2['price_sqf'] = df2['price'] / df2['surface']
df2

,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,highlights,furnishing,amenities,completion_year,floor,price_sqf
listing_id,,,,,,,,,,,,,,,,
5825610,https://www.bayut.com/property/details-5825610...,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,1,815.0,25.177226,55.308851,"Vacant Now, Spacious Layout, 1 Bedroom",NaN,[Balcony or Terrace],0,,1820.580368
5933491,https://www.bayut.com/property/details-5933491...,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,2,1141.0,25.060612,55.200228,"Kitchen Appliances, Pool View, Best Investment",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,,1095.530237
5933545,https://www.bayut.com/property/details-5933545...,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,2,618.0,25.064736,55.207565,"Brand New, Good Investment, Vacant And Ready ...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,,1051.779935
5933677,https://www.bayut.com/property/details-5933677...,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,2,933.0,25.063390,55.202050,"Unfurnished 1BHK + Study Room, Private Garden,...",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,,793.140407
5929393,https://www.bayut.com/property/details-5929393...,Golden Mile 6,Palm Jumeirah,Golden Mile,2500000,2.0,2,1794.0,25.109089,55.144157,"Exclusive, Sea & Park View, Upgraded",Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...",0,,1393.534002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4659412,https://www.bayut.com/property/details-4659412...,NaN,Dubai Residence Complex,Skycourts Towers,369000,1.0,2,734.0,25.091303,55.385476,"New listi0ng, No Commission, High Floor, Vacant",Unfurnished,"[Balcony or Terrace, Parking Spaces, Swimming ...",2010,,502.724796
4513630,https://www.bayut.com/property/details-4513630...,Jumeirah Living,World Trade Centre,World Trade Centre Residence,1850960,2.0,2,1851.0,25.226713,55.289187,"Exclusive, Luxurious Duplex, Open Kitchen",NaN,"[Floor, Steam Room, View, Centrally Air-Condit...",0,,999.978390
4408791,https://www.bayut.com/property/details-4408791...,Icon Tower 1,Jumeirah Lake Towers (JLT),Icon Tower,900000,2.0,3,1098.0,25.064754,55.138866,"Covered Parking, Store & Laundry Room, Wardrobes",Unfurnished,"[Balcony or Terrace, Parking Spaces, Swimming ...",2009,,819.672131


In [9]:
highlights = df2[(df2['highlights'] != '')]['highlights']

h_list = []

for h in highlights:
    if h:
        h_list.append(h)
            
unique_h = list(dict.fromkeys(h_list))
print(unique_h)

['Vacant Now, Spacious Layout, 1 Bedroom', 'Kitchen Appliances, Pool View, Best Investment', 'Brand New, Good Investment,  Vacant And Ready To Move', 'Unfurnished 1BHK + Study Room, Private Garden, Hot Deal', 'Exclusive, Sea & Park View, Upgraded', 'Available and Vacant, Spacious 2 Bedroom + Maid', 'Pool View, Vacant Studio, Without Parking', 'Superb condition, Vastu, Near Metro Station', 'Enchanting 2 BHK for SALE, Grab the Beneficial Deal, Upgraded Living Standard,', 'Partial Sea View, High Floor, 1 Bedroom', 'Upgraded,, C Type,, Vacant on Transfer', 'Spacious I Best Layout in The Building I Bright', 'Upgraded, Amazing Views, Two Bed Potential', 'Luxury Design l Fully Furnished l Spacious Unit', 'Ready To Move, Best Unit, Maid room, Price is Negotiable.', 'Tenanted till Sept I Vacant on Transfer', 'Panoramic Marina Views, High Floor, Luxury Living', 'Spacious Layout,Pool, Park View,Motivated Seller', 'Vacant and ready to move, Boulevard view', 'Vacant I Fully Furnished I Luxury Livin

In [10]:
views = ['view', 'views', 'seaview', 'overlooking', 'facing']
luxury = ['luxury', 'luxurious', 'luxuriously', 'premium']
discounted = ['reduced price', 'priced reduced', 'price reduction', 'discounted price', 'negotiable', \
              'price drop', 'price dropped']
cheap = ['affordable', 'cheap', 'cheapest', 'well-priced', 'well priced', 'economical', 'best price']
distressed = ['distress', 'distressed']
upgraded = ['upgraded', 'partially upgraded', 'fully upgraded', 'semi-upgraded', 'semi upgraded', 'renovated']
condition = ['superb condition', 'brand new', 'new building', 'high quality',\
     'well maintained', 'well-maintained', 'high-end finishing', 'excellent condition', 'great condition',\
     'pristine condition', 'immaculate condition','amazing condition', 'mint condition', 'bes condition', \
     'best condition', 'good condition', 'new condition', 'perfect condition', 'tip-top condition', \
     'turnkey condition', 'top condition', 'impeccable condition','mint condiition']
investment = ['investment', 'investor deal', 'investors deal', 'roi', 'investor visa', 'high returns']
tenanted = ['tenanted', 'rented', 'tenant']
vacant = ['vacant']
metro = ['close to metro', 'next to metro', 'near metro']
furnished = ['furnished', 'unfurnished', 'semi furnished', 'semi-furnished']
studio = ['studio']

In [11]:
def get_feature(f_list, col_name, col_list):
    for row in df2['highlights'].str.lower():
        if any(x in row for x in f_list):
            col_list.append(1)
        else:
            col_list.append(0)
    
    if len(col_list) == len(df2['highlights']):
        df2[col_name] = col_list

In [12]:
get_feature(views, 'views', [])
get_feature(discounted, 'discounted', [])
get_feature(cheap, 'cheap', [])
get_feature(distressed, 'distressed', [])
get_feature(investment, 'investment', [])
get_feature(tenanted, 'tenanted', [])
get_feature(vacant, 'vacant', [])
get_feature(metro, 'metro', [])
get_feature(furnished, 'furnished', [])
get_feature(studio, 'studio', [])
get_feature(condition, 'condition', [])

In [13]:
# Upgraded:

upgraded_p = []

def get_upgraded():
    for row in df2['highlights'].str.lower():
        row = row.split(', ')
        y = ''
        for x in upgraded:
            if x in row:
                y = x
                break

        upgraded_p.append(y)
        
    if len(upgraded_p) == len(df2['highlights']):
        df2['upgraded'] = upgraded_p

get_upgraded()
df2.head(20)

,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,...,cheap,distressed,investment,tenanted,vacant,metro,furnished,studio,condition,upgraded
listing_id,,,,,,,,,,,,,,,,,,,,,
5825610,https://www.bayut.com/property/details-5825610...,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,1,815.0,25.177226,55.308851,...,0,0,0,0,1,0,0,0,0,
5933491,https://www.bayut.com/property/details-5933491...,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,2,1141.0,25.060612,55.200228,...,0,0,1,0,0,0,0,0,0,
5933545,https://www.bayut.com/property/details-5933545...,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,2,618.0,25.064736,55.207565,...,0,0,1,0,1,0,0,0,1,
5933677,https://www.bayut.com/property/details-5933677...,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,2,933.0,25.063390,55.202050,...,0,0,0,0,0,0,1,0,0,
5929393,https://www.bayut.com/property/details-5929393...,Golden Mile 6,Palm Jumeirah,Golden Mile,2500000,2.0,2,1794.0,25.109089,55.144157,...,0,0,0,0,0,0,0,0,0,upgraded
5926764,https://www.bayut.com/property/details-5926764...,Ubora Tower 1,Business Bay,U-Bora Tower,1250000,2.0,4,1333.0,25.179956,55.269723,...,0,0,0,0,1,0,0,0,0,
5926896,https://www.bayut.com/property/details-5926896...,The Crescent Tower B,Dubai Production City (IMPZ),The Crescent Towers,250000,NaN,1,499.0,25.033442,55.202640,...,0,0,0,0,1,0,0,1,0,
5925333,https://www.bayut.com/property/details-5925333...,NaN,Dubai Marina,Yacht Bay,949900,1.0,2,746.0,25.071475,55.136792,...,0,0,0,0,0,1,0,0,1,
5918509,https://www.bayut.com/property/details-5918509...,NaN,Al Furjan,East 40,857000,2.0,2,969.0,25.020959,55.140255,...,0,0,0,0,0,0,0,0,0,


In [14]:
# Luxury:

luxury_col = []

def get_luxury():
    for row in df2['highlights'].str.lower():
        row = row.split(', ')
        y = 0
        for x in condition:
            if x in row:
                y = 1
                break

        luxury_col.append(y)
        
    if len(luxury_col) == len(df2['highlights']):
        df2['luxury'] = luxury_col

get_luxury()
df2.head(20)

,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,...,distressed,investment,tenanted,vacant,metro,furnished,studio,condition,upgraded,luxury
listing_id,,,,,,,,,,,,,,,,,,,,,
5825610,https://www.bayut.com/property/details-5825610...,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,1,815.0,25.177226,55.308851,...,0,0,0,1,0,0,0,0,,0
5933491,https://www.bayut.com/property/details-5933491...,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,2,1141.0,25.060612,55.200228,...,0,1,0,0,0,0,0,0,,0
5933545,https://www.bayut.com/property/details-5933545...,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,2,618.0,25.064736,55.207565,...,0,1,0,1,0,0,0,1,,1
5933677,https://www.bayut.com/property/details-5933677...,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,2,933.0,25.063390,55.202050,...,0,0,0,0,0,1,0,0,,0
5929393,https://www.bayut.com/property/details-5929393...,Golden Mile 6,Palm Jumeirah,Golden Mile,2500000,2.0,2,1794.0,25.109089,55.144157,...,0,0,0,0,0,0,0,0,upgraded,0
5926764,https://www.bayut.com/property/details-5926764...,Ubora Tower 1,Business Bay,U-Bora Tower,1250000,2.0,4,1333.0,25.179956,55.269723,...,0,0,0,1,0,0,0,0,,0
5926896,https://www.bayut.com/property/details-5926896...,The Crescent Tower B,Dubai Production City (IMPZ),The Crescent Towers,250000,NaN,1,499.0,25.033442,55.202640,...,0,0,0,1,0,0,1,0,,0
5925333,https://www.bayut.com/property/details-5925333...,NaN,Dubai Marina,Yacht Bay,949900,1.0,2,746.0,25.071475,55.136792,...,0,0,0,0,1,0,0,1,,1
5918509,https://www.bayut.com/property/details-5918509...,NaN,Al Furjan,East 40,857000,2.0,2,969.0,25.020959,55.140255,...,0,0,0,0,0,0,0,0,,0


In [15]:
df2['luxury'].value_counts()

0    27172
1     1893
Name: luxury, dtype: int64

In [16]:
df2['studio'].value_counts()

0    27265
1     1800
Name: studio, dtype: int64

In [17]:
df2[(df2['beds'].isnull()) & (df2['studio'] == 1)]

,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,...,distressed,investment,tenanted,vacant,metro,furnished,studio,condition,upgraded,luxury
listing_id,,,,,,,,,,,,,,,,,,,,,
5926896,https://www.bayut.com/property/details-5926896...,The Crescent Tower B,Dubai Production City (IMPZ),The Crescent Towers,250000,NaN,1,499.0,25.033442,55.202640,...,0,0,0,1,0,0,1,0,,0
5892751,https://www.bayut.com/property/details-5892751...,Carson Tower A,DAMAC Hills,Carson - The Drive,490000,NaN,1,404.0,25.024358,55.242921,...,0,0,0,0,0,0,1,1,,0
5818854,https://www.bayut.com/property/details-5818854...,Reehan 2,Old Town,Reehan,1300000,NaN,1,606.0,25.189294,55.277241,...,0,0,0,1,0,0,1,0,,0
5871516,https://www.bayut.com/property/details-5871516...,Golf Promenade 2,DAMAC Hills,Golf Promenade,449995,NaN,1,444.0,25.013636,55.250945,...,0,0,0,0,0,1,1,0,,0
5780967,https://www.bayut.com/property/details-5780967...,Murjan 2,Jumeirah Beach Residence (JBR),Murjan,1150000,NaN,1,713.0,25.081989,55.138959,...,0,0,0,0,0,0,1,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5553815,https://www.bayut.com/property/details-5553815...,NaN,Downtown Jebel Ali,Azizi Aura,379000,NaN,1,366.0,24.978431,55.091881,...,0,0,0,0,1,0,1,0,,0
5553499,https://www.bayut.com/property/details-5553499...,Skycourts Tower A,Dubai Residence Complex,Skycourts Towers,275000,NaN,1,441.0,25.092365,55.385187,...,0,0,0,0,0,0,1,0,,0
5520095,https://www.bayut.com/property/details-5520095...,NaN,International City,Spain Cluster,250000,NaN,1,670.0,25.163523,55.397440,...,0,0,0,0,0,0,1,0,,0


In [18]:
df2.loc[(df2['beds'].isnull()) & (df2['studio'] == 1), 'beds'] = 0
df2

,URL,building,district,neighborhood,price,beds,baths,surface,lat,long,...,distressed,investment,tenanted,vacant,metro,furnished,studio,condition,upgraded,luxury
listing_id,,,,,,,,,,,,,,,,,,,,,
5825610,https://www.bayut.com/property/details-5825610...,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,1,815.0,25.177226,55.308851,...,0,0,0,1,0,0,0,0,,0
5933491,https://www.bayut.com/property/details-5933491...,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,2,1141.0,25.060612,55.200228,...,0,1,0,0,0,0,0,0,,0
5933545,https://www.bayut.com/property/details-5933545...,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,2,618.0,25.064736,55.207565,...,0,1,0,1,0,0,0,1,,1
5933677,https://www.bayut.com/property/details-5933677...,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,2,933.0,25.063390,55.202050,...,0,0,0,0,0,1,0,0,,0
5929393,https://www.bayut.com/property/details-5929393...,Golden Mile 6,Palm Jumeirah,Golden Mile,2500000,2.0,2,1794.0,25.109089,55.144157,...,0,0,0,0,0,0,0,0,upgraded,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4659412,https://www.bayut.com/property/details-4659412...,NaN,Dubai Residence Complex,Skycourts Towers,369000,1.0,2,734.0,25.091303,55.385476,...,0,0,0,1,0,0,0,0,,0
4513630,https://www.bayut.com/property/details-4513630...,Jumeirah Living,World Trade Centre,World Trade Centre Residence,1850960,2.0,2,1851.0,25.226713,55.289187,...,0,0,0,0,0,0,0,0,,0
4408791,https://www.bayut.com/property/details-4408791...,Icon Tower 1,Jumeirah Lake Towers (JLT),Icon Tower,900000,2.0,3,1098.0,25.064754,55.138866,...,0,0,0,0,0,0,0,0,,0


In [19]:
df2 = df2.drop('studio', axis=1)

In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29065 entries, 5825610 to 4178490
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   URL              29065 non-null  object 
 1   building         15918 non-null  object 
 2   district         29065 non-null  object 
 3   neighborhood     28708 non-null  object 
 4   price            29065 non-null  int64  
 5   beds             27438 non-null  float64
 6   baths            29065 non-null  int64  
 7   surface          29065 non-null  float64
 8   lat              29065 non-null  float64
 9   long             29065 non-null  float64
 10  highlights       29065 non-null  object 
 11  furnishing       17120 non-null  object 
 12  amenities        29065 non-null  object 
 13  completion_year  29065 non-null  object 
 14  floor            29065 non-null  object 
 15  price_sqf        29065 non-null  float64
 16  views            29065 non-null  int64  
 17  disc

In [21]:
df2.to_csv('../data/bayut/cleaned/ready_flats_clean.csv')

In [22]:
df2['district'].value_counts()

Downtown Dubai                   4131
Dubai Marina                     3760
Business Bay                     2411
Jumeirah Village Circle (JVC)    2355
Palm Jumeirah                    1825
                                 ... 
Jebel Ali                           2
City of Arabia                      2
Nad Al Sheba                        2
Al Safa                             1
Ibn Battuta Gate                    1
Name: district, Length: 75, dtype: int64

In [25]:
df_lux = df2.copy()
df_lux['price'] = df_lux['price'] / 1000
df_lux[df_lux['luxury']==1]['price'].describe()

count     1893.000000
mean      1704.081769
std       2435.642365
min        215.000000
25%        741.850000
50%       1245.000000
75%       2000.000000
max      83200.000000
Name: price, dtype: float64